### Connect to/Create database

In [196]:
import sqlite3 as lite
import pandas as pd

try:
    con = lite.connect('cities.db')
    cur = con.cursor()
except Exception as e:
    print(type(e), '\n', e)
else:
    print('Connected to database \'cities.db\'')

Connected to database 'cities.db'


### Create tables in database
Notes: 
- We allow a city to have several industries, although we define only one main industry in the City class. Later we can add more industries per city.
- Tables 'cities' and 'weather' could be merged into a unique table but we keep them separated here for practice purposes

In [197]:
with con:
    tables = ['cities', 'weather', 'industry'] #tables to delete
    for table in tables:
        try:
            cur.execute('DROP TABLE {};'.format(table))
        except:
            print('Could not delete table \'{}\''.format(table))
        else:
            print("Table '{}' deleted".format(table))

Table 'cities' deleted
Table 'weather' deleted
Table 'industry' deleted


In [198]:
tables_list = ['cities', 'weather', 'industry']
with con:
    command = {}
    # cities
    command['cities'] = '''CREATE TABLE IF NOT EXISTS cities (
    name TEXT PRIMARY KEY,
    state TEXT);'''
    # weather
    command['weather'] = '''CREATE TABLE IF NOT EXISTS weather (
    city TEXT PRIMARY KEY,
    high_temp INT,
    low_temp INT,
    CHECK(high_temp > low_temp));'''
    # industry
    command['industry'] = '''CREATE TABLE IF NOT EXISTS industry (
    id INTEGER PRIMARY KEY,
    city TEXT,
    state TEXT,
    industry TEXT,
    FOREIGN KEY (city) REFERENCES cities(name));'''
    
    for table in ['cities', 'weather', 'industry']:
        try:
            cur.execute(command[table])
        except Exception as e:
            print(type(e), '\n', e)
        else:
            print('Table {} created/already exists'.format(table))

Table cities created/already exists
Table weather created/already exists
Table industry created/already exists


### Create class for cities

In [199]:
class City():
    '''A simple class for location, weather, and industry in a city'''
    
    def __init__(self, name, state, high_temp=None, low_temp=None, industry='N/A'):
        '''Initialize city attributes'''
        self.name = name
        self.state = state
        self.high_temp = high_temp
        self.low_temp = low_temp
        self.industry = industry
        
    def __str__(self):
        if self.industry == 'N/A':
            str1 = '{} is located in the state of {} and its main industry is unknown.'.format(
            self.name, self.state)
        else:
            str1 = '{} is located in the state of {} and its main industry is {}.'.format(
                self.name, self.state, self.industry)
        str2 = 'Average high and low temperatures are {} and {}, respectively.'.format(
            self.high_temp, self.low_temp)
        if 'None and None' in str2:
            str2 = 'Average high and low temperatures are unknown.'
        elif 'None' in str2:
            str2 = str2.replace('None', 'unknown')
        
        return '{}\n{}'.format(str1, str2)
        
    def get_name(self):
        return self.name
    
    def get_state(self):
        return self.state
        
    def get_temps(self):
        return (self.high_temp, self.low_temp)
    
#     def set_temps(self, high_temp=self.high_temp, low_temp=self.low_temp):
    def set_temps(self, high_temp, low_temp):
        '''Set high and low temperatures'''
        self.high_temp = high_temp
        self.low_temp = low_temp
        
    def get_industry(self):
        return self.industry
    
    def set_industry(self, industry):
        '''Set the industry of the city'''
        self.industry = industry

### Create cities

In [200]:
# print(help(City))
houston = City('Houston', 'TX', 90, 45, 'Oil and Gas')
new_york = City('New York', 'NY', 80, 30, 'Finance')
san_francisco = City('San Fransisco', 'CA', 83, 45, 'Hi-Tech')
los_angeles = City('Los Angeles', 'CA', industry='Entertainment')
seattle = City('Seattle', 'WA', 78, 29, 'Hi-Tech')
washington = City('Washington', 'DC', 82, 33, 'Government')
denver = City('Denver', 'CO', low_temp=27)

city_list = [houston, new_york, san_francisco, los_angeles, seattle,
            washington, denver]

In [201]:
print(houston)
print()
print(seattle)
print()
print(los_angeles)
print()
print(denver)

Houston is located in the state of TX and its main industry is Oil and Gas.
Average high and low temperatures are 90 and 45, respectively.

Seattle is located in the state of WA and its main industry is Hi-Tech.
Average high and low temperatures are 78 and 29, respectively.

Los Angeles is located in the state of CA and its main industry is Entertainment.
Average high and low temperatures are unknown.

Denver is located in the state of CO and its main industry is unknown.
Average high and low temperatures are unknown and 27, respectively.


### Save cities information in databases
Reminder:
- cities values: (name TEXT PRIMARY KEY, state TEXT)
- weather values: (city TEXT PRIMARY KEY, high_temp INT, low_temp INT)
- industry values: (id INT PRIMARY KEY, city TEXT, state TEXT, industry TEXT)

In [202]:
with con:
    for city in city_list:
        command['cities'] = "INSERT INTO cities (name, state) VALUES ('{}', '{}');".format(
                             city.get_name(), city.get_state())
        high_t = city.get_temps()[0] if city.get_temps()[0] is not None else 'NULL' # otherwise issue when inserting None
        low_t = city.get_temps()[1] if city.get_temps()[1] is not None else 'NULL'
        command['weather'] = "INSERT INTO weather (city, high_temp, low_temp) VALUES ('{}', {}, {});".format(
                              city.get_name(), high_t, low_t)
        command['industry'] = "INSERT INTO industry (city, state, industry) VALUES ('{}', '{}', '{}')".format(
                               city.get_name(), city.get_state(), city.get_industry())
        
        for table in tables_list:
            cur.execute(command[table])

##### Check if information was properly inserted

In [203]:
df_cities = pd.read_sql('SELECT * FROM cities;', con)
print(df_cities, '\n')
df_weather = pd.read_sql('SELECT * FROM weather;', con)
print(df_weather, '\n')
df_industry = pd.read_sql('SELECT * FROM industry;', con)
print(df_industry)

            name state
0        Houston    TX
1       New York    NY
2  San Fransisco    CA
3    Los Angeles    CA
4        Seattle    WA
5     Washington    DC
6         Denver    CO 

            city  high_temp  low_temp
0        Houston       90.0      45.0
1       New York       80.0      30.0
2  San Fransisco       83.0      45.0
3    Los Angeles        NaN       NaN
4        Seattle       78.0      29.0
5     Washington       82.0      33.0
6         Denver        NaN      27.0 

   id           city state       industry
0   1        Houston    TX    Oil and Gas
1   2       New York    NY        Finance
2   3  San Fransisco    CA        Hi-Tech
3   4    Los Angeles    CA  Entertainment
4   5        Seattle    WA        Hi-Tech
5   6     Washington    DC     Government
6   7         Denver    CO            N/A


##### Get everything in a single dataframe (1 industry per city at this point)

In [211]:
command = '''SELECT c.name AS name, c.state as state, j1.high_temp as avg_high_temp,
                    j1.low_temp as avg_low_temp, j1.industry as main_industry FROM cities c JOIN
             (SELECT w.city, w.high_temp, w.low_temp, i.industry FROM weather w JOIN
              industry i ON w.city = i.city) j1
             ON c.name = j1.city'''
df_all = pd.read_sql(command, con)
print(df_all)

            name state  avg_high_temp  avg_low_temp  main_industry
0        Houston    TX           90.0          45.0    Oil and Gas
1       New York    NY           80.0          30.0        Finance
2  San Fransisco    CA           83.0          45.0        Hi-Tech
3    Los Angeles    CA            NaN           NaN  Entertainment
4        Seattle    WA           78.0          29.0        Hi-Tech
5     Washington    DC           82.0          33.0     Government
6         Denver    CO            NaN          27.0            N/A


### Add a new colunm 'population' to the table 'cities'